# CS2203 Lab 4
Problem Statement : You are required to design and implement the Convolutional Neural Network (CNN) architecture
depicted in Figure using TensorFlow/Keras Functional API.

In [19]:
# Importing required libraries
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers, Model

In [20]:
# Input data
input_data = np.array([
    [1, 2, 3, 0, 1],
    [0, 1, 2, 3, 0],
    [3, 0, 1, 2, 1],
    [1, 2, 3, 0, 0],
    [0, 1, 0, 1, 2]
], dtype = np.float32).reshape((1, 5, 5, 1))

print("Input shape : ", input_data.shape)

Input shape :  (1, 5, 5, 1)


# Architecture

In [21]:
# Input Layer
inputs = layers.Input(shape = (5, 5, 1), name="Input_Layer")

# Initial Block (Conv2D (3*3 + Leaky Relu))
x = layers.Conv2D(
    filters = 1,
    kernel_size = (3,3),
    padding = 'same',
    kernel_initializer = 'ones',
    use_bias = False,
    name = "Initial_Conv"
)(inputs)

x = layers.LeakyReLU(alpha = 0.01, name="Initial_LeakyRelu")(x)

# Residual Block

# Part A : Skip Connection
skip = x

# Part B : Dilated Convolutions (r = 3->2->1)

# r = 3
b = layers.Conv2D(
    filters = 1,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 3,
    kernel_initializer = 'ones',
    use_bias = False,
    name = "Dilated_Conv_r3"
)(x)

b = layers.LeakyReLU(alpha = 0.01, name="Dilated_Conv_r3_LeakyRelu")(b)

# r = 2
b = layers.Conv2D(
    filters = 1,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 2,
    kernel_initializer = 'ones',
    use_bias = False,
    name = "Dilated_Conv_r2"
)(b)

b = layers.LeakyReLU(alpha = 0.01, name="Dilated_Conv_r2_LeakyRelu")(b)

# r = 1
b = layers.Conv2D(
    filters = 1,
    kernel_size = (3,3),
    padding = 'same',
    dilation_rate = 1,
    kernel_initializer = 'ones',
    use_bias = False,
    name = "Dilated_Conv_r1"
)(b)

b = layers.LeakyReLU(alpha = 0.01, name="Dilated_Conv_r1_LeakyRelu")(b)


# Residual Addition
residual_out = layers.Add(name = "Residual_Addition")([skip, b])

# Intermediate Conv Layer
intermediate = layers.Conv2D(
    filters = 1,
    kernel_size = (3,3),
    padding = 'same',
    kernel_initializer = 'ones',
    use_bias = False,
    name = "Intermediate_Conv"
)(residual_out)

# Parallel Pooling
avg_pool = layers.AveragePooling2D(
    pool_size = (2, 2),
    strides = 2,
    name = "Avg_Pooling"
)(intermediate)

max_pool = layers.MaxPool2D(
    pool_size = (2, 2),
    strides = 2,
    name = "Max_Pooling"
)(intermediate)

# Final Output
outputs = layers.Add(name = "Final_Output")([avg_pool, max_pool])

model = Model(inputs = inputs, outputs = outputs)

In [22]:
model.summary()

Model: "functional_42"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ Input_Layer         │ (None, 5, 5, 1)   │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_Conv        │ (None, 5, 5, 1)   │          9 │ Input_Layer[0][0] │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Initial_LeakyRelu   │ (None, 5, 5, 1)   │          0 │ Initial_Conv[0][… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r3     │ (None, 5, 5, 1)   │          9 │ Initial_LeakyRel… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r3_Le… │ (None, 5, 5, 1)   │          0 │ Dilated_Conv_r3[… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r2     │ (None, 5, 5, 1)   │          9 │ Dilated_Conv_r3_… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r2_Le… │ (None, 5, 5, 1)   │          0 │ Dilated_Conv_r2[… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r1     │ (None, 5, 5, 1)   │          9 │ Dilated_Conv_r2_… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Dilated_Conv_r1_Le… │ (None, 5, 5, 1)   │          0 │ Dilated_Conv_r1[… │
│ (LeakyReLU)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Residual_Addition   │ (None, 5, 5, 1)   │          0 │ Initial_LeakyRel… │
│ (Add)               │                   │            │ Dilated_Conv_r1_… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Intermediate_Conv   │ (None, 5, 5, 1)   │          9 │ Residual_Additio… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Avg_Pooling         │ (None, 2, 2, 1)   │          0 │ Intermediate_Con… │
│ (AveragePooling2D)  │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Max_Pooling         │ (None, 2, 2, 1)   │          0 │ Intermediate_Con… │
│ (MaxPooling2D)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Final_Output (Add)  │ (None, 2, 2, 1)   │          0 │ Avg_Pooling[0][0… │
│                     │                   │            │ Max_Pooling[0][0] │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 45 (180.00 B)

 Trainable params: 45 (180.00 B)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Printing Output
output = model.predict(input_data)

print("Final Output Feature Map:")
print(output)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step
Final Output Feature Map:
[[[[13836.5 ]
   [17082.75]]

  [[17054.25]
   [21165.  ]]]]


In [24]:
# Function to print feature maps after each layer
def pretty_print_layer_output(model, layer_name, input_data):
    intermediate_model = tf.keras.Model(
        inputs=model.input,
        outputs=model.get_layer(layer_name).output
    )

    output = intermediate_model.predict(input_data, verbose=0)

    # Remove batch & channel dimensions → (H, W)
    matrix = np.squeeze(output)

    print(f"\n==============================")
    print(f"Layer: {layer_name}")
    print(f"Output shape: {output.shape}")
    print("------------------------------")

    for row in matrix:
        print(" ".join(f"{val:6.2f}" for val in row))

# Layers to inspect
layer_names = [
    "Initial_Conv",
    "Initial_LeakyRelu",

    "Dilated_Conv_r3",
    "Dilated_Conv_r3_LeakyRelu",

    "Dilated_Conv_r2",
    "Dilated_Conv_r2_LeakyRelu",

    "Dilated_Conv_r1",
    "Dilated_Conv_r1_LeakyRelu",

    "Residual_Addition",
    "Intermediate_Conv",

    "Avg_Pooling",
    "Max_Pooling",

    "Final_Output"
]

# Print all layer outputs
for layer in layer_names:
    pretty_print_layer_output(model, layer, input_data)



Layer: Initial_Conv
Output shape: (1, 5, 5, 1)
------------------------------
  4.00   9.00  11.00   9.00   4.00
  7.00  13.00  14.00  13.00   7.00
  7.00  13.00  14.00  12.00   6.00
  7.00  11.00  10.00  10.00   6.00
  4.00   7.00   7.00   6.00   3.00

Layer: Initial_LeakyRelu
Output shape: (1, 5, 5, 1)
------------------------------
  4.00   9.00  11.00   9.00   4.00
  7.00  13.00  14.00  13.00   7.00
  7.00  13.00  14.00  12.00   6.00
  7.00  11.00  10.00  10.00   6.00
  4.00   7.00   7.00   6.00   3.00

Layer: Dilated_Conv_r3
Output shape: (1, 5, 5, 1)
------------------------------
 30.00  30.00  21.00  30.00  30.00
 30.00  30.00  21.00  30.00  30.00
 19.00  19.00  14.00  19.00  19.00
 30.00  30.00  21.00  30.00  30.00
 30.00  30.00  21.00  30.00  30.00

Layer: Dilated_Conv_r3_LeakyRelu
Output shape: (1, 5, 5, 1)
------------------------------
 30.00  30.00  21.00  30.00  30.00
 30.00  30.00  21.00  30.00  30.00
 19.00  19.00  14.00  19.00  19.00
 30.00  30.00  21.00  30.00  30.0